# Importing all the libraries required for training the model.

In [1]:
# Title - StackOverflow Tag Prediction
# Authors - Mayur Reddy Sangepu, Ajay Yadav, Anurag Kacham, Hima Bindu Krovvidi

import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle
import matplotlib.pyplot as plt
from collections import Counter
import mysql.connector as sql
from mysql.connector import Error

#questions_df = pd.read_csv("/Users/mayurreddysangepu/Desktop/Group3_Project_Phase2/SourceCode/Sample_Questions.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anuragkacham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anuragkacham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anuragkacham/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anuragkacham/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Loading data from MySQL Workbench

In [2]:
try:
    conn = sql.connect(host='localhost', database='stackoverflow', user='root', password='rootroot')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        query = ("Select id, accepted_answer_id, answer_count, comment_count, " 
                       "creation_date, favorite_count, owner_user_id, related_posts, " 
                       "score, tags, title, view_count from stackoverflow_questions "
                        " where id between %s AND %s")
        start = 1
        end = 50000
        cursor.execute(query, (start, end))
        data = cursor.fetchall()
        print(len(data))
        if len(data) > 0:
            questions_df = pd.DataFrame(data, columns=["_id", "accepted_answer_id", "answer_count", "comment_count",  
                       "creation_date", "favorite_count", "owner_user_id", "related_posts",  
                       "score", "tags", "title", "view_count"])
        while True:
            start = start + 50000
            end = end + 50000
            cursor.execute(query, (start, end))
            data = cursor.fetchall()
            if len(data)>0:
                temp_df = pd.DataFrame(data, index= range (len(questions_df),len(questions_df)+len(data)), columns=["_id", "accepted_answer_id", "answer_count", "comment_count",  
                      "creation_date", "favorite_count", "owner_user_id", "related_posts",  
                      "score", "tags", "title", "view_count"])
                ques_df = [questions_df, temp_df]
                questions_df = pd.concat(ques_df)
            else:
                break
        print("Data Loaded")
        cursor.close()
        conn.close()
            
except Error as e:
        print("Error while connecting to MySQL", e)
        cursor.close()
        conn.close()


You're connected to database:  ('stackoverflow',)
68
Data Loaded


# Dispalying loaded data

In [3]:
questions_df

,_id,accepted_answer_id,answer_count,comment_count,creation_date,favorite_count,owner_user_id,related_posts,score,tags,title,view_count
0,10,7596.0,NaN,NaN,NaT,NaN,576436.0,None,NaN,None,How to make an awesome DBSI Project,NaN
1,174,NaN,7.0,4.0,2008-08-01 18:33:48,7.0,96.0,[564650],83.0,"[""html"",""c#"",""printing"",""web-services""]",How do I print an HTML document from a web ser...,16553.0
2,371,396.0,24.0,0.0,2008-08-02 08:19:18,317.0,49.0,[42200904],452.0,"[""email"",""email-spam""]",How do you make sure email you send programmat...,110617.0
3,549,477578.0,11.0,16.0,2008-08-02 19:51:50,6715.0,136.0,[14907581],5522.0,"[""language-agnostic"",""forms"",""security"",""authe...",The definitive guide to form-based website aut...,640732.0
4,622,623.0,30.0,3.0,2008-08-03 05:45:21,24.0,184.0,"[2211990,13894417]",63.0,"[""algorithm"",""performance"",""primes""]",Most efficient code for the first 10000 prime ...,68443.0
...,...,...,...,...,...,...,...,...,...,...,...,...
440055,70231258,NaN,0.0,2.0,2021-12-05 03:27:01,NaN,7590347.0,[26621746],1.0,"[""rust""]",Is it possible to read and write at the same t...,15.0
440056,70231449,NaN,0.0,0.0,2021-12-05 04:18:24,NaN,355456.0,[60197903],0.0,"[""android-databinding""]",Two-way binding of every Observable field caus...,7.0
440057,70231476,NaN,0.0,2.0,2021-12-05 04:24:39,NaN,17592241.0,[19409717],0.0,"[""excel"",""vba"",""autofill""]",Autofill down to the last row,11.0
440058,70231519,NaN,0.0,0.0,2021-12-05 04:37:14,NaN,8748688.0,[44945766],0.0,"[""html"",""javascript"",""css"",""angular""]",how is possible add javascript file in angular...,11.0


# Displaying information about type of the loaded data


In [4]:
questions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440060 entries, 0 to 440059
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   _id                 440060 non-null  int64         
 1   accepted_answer_id  155150 non-null  float64       
 2   answer_count        440059 non-null  float64       
 3   comment_count       440059 non-null  float64       
 4   creation_date       440059 non-null  datetime64[ns]
 5   favorite_count      107323 non-null  float64       
 6   owner_user_id       432213 non-null  float64       
 7   related_posts       440059 non-null  object        
 8   score               440059 non-null  float64       
 9   tags                440059 non-null  object        
 10  title               440060 non-null  object        
 11  view_count          440059 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 40.3+ MB


# clean_tags function cleans the tags and formats them as a list of tags 

In [5]:
questions_df['tags_list'] = None
print(len(questions_df.tags))
def clean_tags():
    for i in range(0,len(questions_df.tags)):
        if type(questions_df.tags[i]) == str:
            tmp = questions_df.tags[i].replace(",","")
            tmp = tmp.replace("[","")
            tmp = tmp.replace("]","")
            tmp = tmp.replace(" ","")
            tmp = tmp.split("\"")
            test_list = list(filter(None, tmp))
            questions_df.tags_list[i] = test_list

440060


In [6]:
clean_tags()

/var/folders/nn/034qbjws18j5cqqw317jpgsw0000gn/T/ipykernel_8039/1785244.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions_df.tags_list[i] = test_list


In [7]:
questions_df

,_id,accepted_answer_id,answer_count,comment_count,creation_date,favorite_count,owner_user_id,related_posts,score,tags,title,view_count,tags_list
0,10,7596.0,NaN,NaN,NaT,NaN,576436.0,None,NaN,None,How to make an awesome DBSI Project,NaN,None
1,174,NaN,7.0,4.0,2008-08-01 18:33:48,7.0,96.0,[564650],83.0,"[""html"",""c#"",""printing"",""web-services""]",How do I print an HTML document from a web ser...,16553.0,"[html, c#, printing, web-services]"
2,371,396.0,24.0,0.0,2008-08-02 08:19:18,317.0,49.0,[42200904],452.0,"[""email"",""email-spam""]",How do you make sure email you send programmat...,110617.0,"[email, email-spam]"
3,549,477578.0,11.0,16.0,2008-08-02 19:51:50,6715.0,136.0,[14907581],5522.0,"[""language-agnostic"",""forms"",""security"",""authe...",The definitive guide to form-based website aut...,640732.0,"[language-agnostic, forms, security, authentic..."
4,622,623.0,30.0,3.0,2008-08-03 05:45:21,24.0,184.0,"[2211990,13894417]",63.0,"[""algorithm"",""performance"",""primes""]",Most efficient code for the first 10000 prime ...,68443.0,"[algorithm, performance, primes]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440055,70231258,NaN,0.0,2.0,2021-12-05 03:27:01,NaN,7590347.0,[26621746],1.0,"[""rust""]",Is it possible to read and write at the same t...,15.0,[rust]
440056,70231449,NaN,0.0,0.0,2021-12-05 04:18:24,NaN,355456.0,[60197903],0.0,"[""android-databinding""]",Two-way binding of every Observable field caus...,7.0,[android-databinding]
440057,70231476,NaN,0.0,2.0,2021-12-05 04:24:39,NaN,17592241.0,[19409717],0.0,"[""excel"",""vba"",""autofill""]",Autofill down to the last row,11.0,"[excel, vba, autofill]"
440058,70231519,NaN,0.0,0.0,2021-12-05 04:37:14,NaN,8748688.0,[44945766],0.0,"[""html"",""javascript"",""css"",""angular""]",how is possible add javascript file in angular...,11.0,"[html, javascript, css, angular]"


# Creating new column as clean_tags where we are considering 'python', 'r', 'c#', 'java', 'android', 'html', 'kotlin', 'c', 'c++' language question with tags

In [8]:
req_tags = ['python', 'r', 'c#', 'java', 'android', 'html', 'kotlin', 'c', 'c++']

def clean_tags(tag):
    clean_tag = []
    for t in tag:
        if t in req_tags:
            clean_tag.append(t)
        return clean_tag 

In [9]:
questions_df["clean_tags"]=""
def create_clean_tags():
    for i in range(0,len(questions_df["tags_list"])):
        if (type(questions_df["tags_list"][i]) == list):
            if(len(clean_tags(questions_df["tags_list"][i])) > 0): 
                questions_df.clean_tags[i] = clean_tags(questions_df["tags_list"][i])[0]

In [10]:
create_clean_tags()

/var/folders/nn/034qbjws18j5cqqw317jpgsw0000gn/T/ipykernel_8039/1621884911.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions_df.clean_tags[i] = clean_tags(questions_df["tags_list"][i])[0]


In [11]:
len(questions_df)

440060

In [12]:
indexes = []
for i in range(0,len(questions_df)):
    if(questions_df['clean_tags'][i] == ''):
        indexes.append(questions_df['_id'][i])

In [13]:
questions_df_copy = questions_df.copy()

In [14]:
indexes

[10,
 371,
 549,
 622,
 888,
 898,
 1108,
 3224,
 3255,
 3432,
 3859,
 4314,
 4533,
 7694,
 9729,
 11311,
 14278,
 14731,
 15171,
 19353,
 20021,
 22818,
 25969,
 28637,
 28950,
 28982,
 29011,
 29847,
 30058,
 32241,
 32414,
 33559,
 34249,
 39281,
 39567,
 40112,
 40230,
 41039,
 42446,
 43289,
 44153,
 44394,
 45015,
 45908,
 46586,
 48916,
 50823,
 50900,
 50902,
 51054,
 51165,
 51680,
 54539,
 54851,
 55179,
 57287,
 58190,
 59022,
 59099,
 59327,
 60034,
 60658,
 61110,
 62814,
 65058,
 67093,
 67339,
 67699,
 68283,
 69849,
 70855,
 70947,
 72209,
 74625,
 75614,
 76595,
 77133,
 80357,
 82882,
 84847,
 85116,
 88011,
 88235,
 89465,
 90037,
 94591,
 96759,
 98033,
 98096,
 98205,
 98268,
 98449,
 99553,
 100631,
 100795,
 101316,
 101647,
 101752,
 102049,
 102083,
 104158,
 105522,
 105572,
 109232,
 111436,
 111597,
 112055,
 112727,
 112866,
 113547,
 113780,
 114501,
 114527,
 115159,
 115851,
 116907,
 116988,
 117171,
 117962,
 118100,
 120334,
 124269,
 125400,
 125541,

In [15]:
questions_df_copy = questions_df_copy[ (questions_df_copy['clean_tags'] != '')]

In [16]:
questions_df_copy

,_id,accepted_answer_id,answer_count,comment_count,creation_date,favorite_count,owner_user_id,related_posts,score,tags,title,view_count,tags_list,clean_tags
1,174,NaN,7.0,4.0,2008-08-01 18:33:48,7.0,96.0,[564650],83.0,"[""html"",""c#"",""printing"",""web-services""]",How do I print an HTML document from a web ser...,16553.0,"[html, c#, printing, web-services]",html
5,773,7286.0,14.0,1.0,2008-08-03 18:27:10,241.0,207.0,"[2154249,5695208,44790869]",594.0,"[""python"",""itertools""]",How do I use itertools.groupby()?,361938.0,"[python, itertools]",python
9,2540,2697.0,5.0,0.0,2008-08-05 16:30:37,20.0,136.0,[17447802],48.0,"[""c"",""architecture"",""data-structures""]",Good STL-like library for C,8146.0,"[c, architecture, data-structures]",c
14,4306,4311.0,11.0,0.0,2008-08-07 02:29:58,17.0,522.0,[35985960],56.0,"[""c++"",""oop"",""hash"",""maps"",""data-structures""]",What is the best way to create a sparse array ...,50927.0,"[c++, oop, hash, maps, data-structures]",c++
17,4973,5103.0,14.0,0.0,2008-08-07 17:02:23,7.0,404.0,[211383],39.0,"[""html"",""css""]",Setting a div's height in HTML with CSS,174217.0,"[html, css]",html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440050,70231047,NaN,0.0,1.0,2021-12-05 02:25:15,NaN,15180075.0,[5963269],-1.0,"[""r"",""lubridate""]",How to convert a Date column with lubridate an...,10.0,"[r, lubridate]",r
440051,70231098,NaN,0.0,0.0,2021-12-05 02:37:00,NaN,17495244.0,[49649215],0.0,"[""python"",""import"",""excel"",""image""]",Import images from Excel to Python to create p...,8.0,"[python, import, excel, image]",python
440054,70231232,NaN,0.0,2.0,2021-12-05 03:16:10,NaN,17591974.0,[1931126],0.0,"[""c++"",""pointers""]",clearing a vector of pointers (affecting the d...,17.0,"[c++, pointers]",c++
440058,70231519,NaN,0.0,0.0,2021-12-05 04:37:14,NaN,8748688.0,[44945766],0.0,"[""html"",""javascript"",""css"",""angular""]",how is possible add javascript file in angular...,11.0,"[html, javascript, css, angular]",html


# After preprocesssing / cleaning data saving it into the excel

In [17]:
questions_df_copy.to_excel("questions_tag_clean.xlsx")

In [18]:
test = pd.read_excel("questions_tag_clean.xlsx")

In [19]:
test

,Unnamed: 0,_id,accepted_answer_id,answer_count,comment_count,creation_date,favorite_count,owner_user_id,related_posts,score,tags,title,view_count,tags_list,clean_tags
0,1,174,NaN,7,4,2008-08-01 18:33:48,7.0,96.0,[564650],83,"[""html"",""c#"",""printing"",""web-services""]",How do I print an HTML document from a web ser...,16553,"['html', 'c#', 'printing', 'web-services']",html
1,5,773,7286.0,14,1,2008-08-03 18:27:10,241.0,207.0,"[2154249,5695208,44790869]",594,"[""python"",""itertools""]",How do I use itertools.groupby()?,361938,"['python', 'itertools']",python
2,9,2540,2697.0,5,0,2008-08-05 16:30:37,20.0,136.0,[17447802],48,"[""c"",""architecture"",""data-structures""]",Good STL-like library for C,8146,"['c', 'architecture', 'data-structures']",c
3,14,4306,4311.0,11,0,2008-08-07 02:29:58,17.0,522.0,[35985960],56,"[""c++"",""oop"",""hash"",""maps"",""data-structures""]",What is the best way to create a sparse array ...,50927,"['c++', 'oop', 'hash', 'maps', 'data-structures']",c++
4,17,4973,5103.0,14,0,2008-08-07 17:02:23,7.0,404.0,[211383],39,"[""html"",""css""]",Setting a div's height in HTML with CSS,174217,"['html', 'css']",html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199689,440050,70231047,NaN,0,1,2021-12-05 02:25:15,NaN,15180075.0,[5963269],-1,"[""r"",""lubridate""]",How to convert a Date column with lubridate an...,10,"['r', 'lubridate']",r
199690,440051,70231098,NaN,0,0,2021-12-05 02:37:00,NaN,17495244.0,[49649215],0,"[""python"",""import"",""excel"",""image""]",Import images from Excel to Python to create p...,8,"['python', 'import', 'excel', 'image']",python
199691,440054,70231232,NaN,0,2,2021-12-05 03:16:10,NaN,17591974.0,[1931126],0,"[""c++"",""pointers""]",clearing a vector of pointers (affecting the d...,17,"['c++', 'pointers']",c++
199692,440058,70231519,NaN,0,0,2021-12-05 04:37:14,NaN,8748688.0,[44945766],0,"[""html"",""javascript"",""css"",""angular""]",how is possible add javascript file in angular...,11,"['html', 'javascript', 'css', 'angular']",html


# Taking only two columns for training model
## 1. Title : questions posted
## 2. clean_tags : label

In [20]:
test = test.drop(columns=['Unnamed: 0', '_id', 'accepted_answer_id', 'answer_count',
       'comment_count', 'creation_date', 'favorite_count', 'owner_user_id',
       'related_posts', 'score', 'tags', 'view_count', 'tags_list'])

In [21]:
test

,title,clean_tags
0,How do I print an HTML document from a web ser...,html
1,How do I use itertools.groupby()?,python
2,Good STL-like library for C,c
3,What is the best way to create a sparse array ...,c++
4,Setting a div's height in HTML with CSS,html
...,...,...
199689,How to convert a Date column with lubridate an...,r
199690,Import images from Excel to Python to create p...,python
199691,clearing a vector of pointers (affecting the d...,c++
199692,how is possible add javascript file in angular...,html


# Looking at the data we can understand that this is an NLP problem and for solving this problem we have to convert text into number form so that machine can understand.

## We need to remove special character and take all the text in lower case as well as remove the stop word from sentences to get the proper input for training the model


In [22]:
## tokanizer used for splitting sentences in the word form

def sent_to_words(sentences):
    return word_tokenize(sentences)

## lemmatizer transforms the word to its simple form.
# for example, doing -> do, working -> work
lemmatizer = WordNetLemmatizer()
def data_lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for i in texts:
        texts_out.append(lemmatizer.lemmatize(i))
    return texts_out

## Here we are removing the stop words (for example, 'is', 'the', 'in')
stop_words = set(stopwords.words('english'))
def remove_stop_words(sent):
    filtered_sentence = []
    for w in sent:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    return filtered_sentence


## calling all of the above function and using the regular expression to remove special character.
# Building a preprocessing function to clean text
def preprocess_corpus(x):

    # Adding a space between words and punctation
    x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
    x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)

    # converting to lower case
    x = x.lower()

    #correct some acronyms/typos/abbreviations  
    x = re.sub(r"lol", "lots of laughs", x)
    x = re.sub(r"amirite", "am i right", x)
    x = re.sub(r"\b(tho)\b", "though", x)
    x = re.sub(r"\b(ikr)\b", "i know right", x)
    x = re.sub(r"\b(ya|u)\b", "you", x)
    x = re.sub(r"\b(eu)\b", "europe", x)
    x = re.sub(r"\b(da)\b", "the", x)
    x = re.sub(r"\b(dat)\b", "that", x)
    x = re.sub(r"\b(dats)\b", "that is", x)
    x = re.sub(r"\b(cuz)\b", "because", x)
    x = re.sub(r"\b(tbh)\b", "to be honest", x)
    x = re.sub(r"\b(tbf)\b", "to be fair", x)
    x = re.sub(r"faux pas", "mistake", x)
    x = re.sub(r"\b(btw)\b", "by the way", x)
    x = re.sub(r"\b(kinda)\b", "kind of", x)
    x = re.sub(r"\b(bruh)\b", "bro", x)
    x = re.sub(r"\b(w/e)\b", "whatever", x)
    x = re.sub(r"\b(w/)\b", "with", x)
    x = re.sub(r"\b(w/o)\b", "without", x)
    x = re.sub(r"\b(doj)\b", "department of justice", x)

    #replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
    x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
    x = re.sub(r"\b(co+l+)\b", "cool", x)
    x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
    x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
    x = re.sub(r"\b(o+m+g+)\b", "omg", x)
    x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
    x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
    x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
    x = re.sub(r"\b(w+o+w+)\b", "wow", x)
    x = re.sub(r"\b(w+h+y+)\b", "why", x)
    x = re.sub(r"\b(s+o+)\b", "so", x)
    x = re.sub(r"\b(f)\b", "fuck", x)
    x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
    x = re.sub(r"\b(ofc)\b", "of course", x)
    x = re.sub(r"\b(the us)\b", "usa", x)
    x = re.sub(r"\b(gf)\b", "girlfriend", x)
    x = re.sub(r"\b(hr)\b", "human ressources", x)
    x = re.sub(r"\b(mh)\b", "mental health", x)
    x = re.sub(r"\b(idk)\b", "i do not know", x)
    x = re.sub(r"\b(gotcha)\b", "i got you", x)
    x = re.sub(r"\b(y+e+p+)\b", "yes", x)
    x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
    x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
    x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
    x = re.sub(r"\b(o+h+)\b", "oh", x)
    x = re.sub(r"\b(a+h+)\b", "ah", x)
    x = re.sub(r"\b(u+h+)\b", "uh", x)

    # Handling emojis
    x = re.sub(r"<3", " love ", x)
    x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
    x = re.sub(r":\)", " smiling_face ", x)
    x = re.sub(r"^_^", " smiling_face ", x)
    x = re.sub(r"\*_\*", " star_struck ", x)
    x = re.sub(r":\(", " frowning_face ", x)
    x = re.sub(r":\^\(", " frowning_face ", x)
    x = re.sub(r";\(", " frowning_face ", x)
    x = re.sub(r":\/",  " confused_face", x)
    x = re.sub(r";\)",  " wink", x)
    x = re.sub(r">__<",  " unamused ", x)
    x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
    x = re.sub(r"\b(n+a+h+)\b", "no", x)

    # Remove special characters and numbers replace by space + remove double space
    x = re.sub(r"\b([.]{3,})"," dots ", x)
    x = re.sub(r"[^A-Za-z!?_]+"," ", x)
    x = re.sub(r"\b([s])\b *","", x)
    x = re.sub(r" +"," ", x)
    x = re.sub('[^A-Za-z0-9]+', ' ', x)
    x = re.sub(r'[0-9]', ' ', x)
    x = sent_to_words(x)
    x = remove_stop_words(x)
    x = data_lemmatization(x)
    x = ' '.join(x)
    return x

In [23]:
test = test[test['title'] != 0]

In [24]:
test.reset_index(inplace=True)

In [25]:
test

,index,title,clean_tags
0,0,How do I print an HTML document from a web ser...,html
1,1,How do I use itertools.groupby()?,python
2,2,Good STL-like library for C,c
3,3,What is the best way to create a sparse array ...,c++
4,4,Setting a div's height in HTML with CSS,html
...,...,...,...
199681,199689,How to convert a Date column with lubridate an...,r
199682,199690,Import images from Excel to Python to create p...,python
199683,199691,clearing a vector of pointers (affecting the d...,c++
199684,199692,how is possible add javascript file in angular...,html


In [26]:
text = test["title"]

In [27]:
label = test['clean_tags']

In [28]:
text_clean = []
for i in range(0,len(text)):
    text_clean.append(preprocess_corpus(text[i]))

In [29]:
for i in range(0,len(label)):
    x = label[i]
    x = x.replace("+","plus")
#     label.replace("+","plus")
    x = x.replace("#",'hash')
    label[i] = x

/var/folders/nn/034qbjws18j5cqqw317jpgsw0000gn/T/ipykernel_8039/187975501.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[i] = x


# special character in labels is replaced by words

# Training the model using gradient boosting classifier model

In [30]:
def train_model(text,labels):
    X = text
    y = labels
    from sklearn.feature_extraction.text import CountVectorizer
    cv=CountVectorizer(max_features=1500)
    X_train = cv.fit_transform(X).toarray()
    pickle.dump(cv, open('cv.pkl', 'wb'))
    
    from sklearn.ensemble import GradientBoostingClassifier
    gbModel = GradientBoostingClassifier(random_state=42)
    
    gbModel.fit(X_train,y)

    y_pred = gbModel.predict(X_train)
    pickle.dump(gbModel, open('gbModel.pkl', 'wb'))
    print('model created and saved the pickle file')
    
    print('Accuracy score: ',accuracy_score(y,y_pred))
    print('F1-Score Macro:',f1_score(y, y_pred, average='macro'))

# Displaying final result 

In [31]:
train_model(text,label)

model created and saved the pickle file
Accuracy score:  0.6231783900724137
F1-Score Macro: 0.5615837929197642
